In [258]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

import datetime as dt
from dateutil.parser import parse

import glob
from pathlib import Path

## Read Files and Extract Data for one plant

In [341]:
# sample input
dat_dir = 'BCBS'

CH_features = ['_kW/Ton','Load','CDWRT','CDWST','Lift','Ton','_EvapApproach',
               '_CondApproach','Hz','_CompSH','COM1IGV','REFLVL']

Chillers = ['1','2','3','4']

PlantCode = 'BB'

df = ReadFiles(dat_dir, PlantCode, Chillers, CH_features)

Plant BCBS with code BB
opening file: data/BCBS/dss_BCBS_chillers 2017-07.csv
opening file: data/BCBS/dss_BCBS_chillers 2017-11.csv
opening file: data/BCBS/dss_BCBS_chillers 2017-05.csv
opening file: data/BCBS/dss_BCBS_chillers 2017-01.csv
opening file: data/BCBS/dss_BCBS_chillers 2017-03.csv
opening file: data/BCBS/dss_BCBS_chillers 2017-09.csv
total data points: (105112, 208) 

removing points where Load=0,kW/Ton=0 and kW/Ton>1 

data/BB1_chiller.csv
data points removed for Startup 526
replacing Lift with DTLift
no CH1Hz column for this chiller
replacing COM1IGV with IGV
replacing Lift with DTLift
no CH1Hz column for this chiller
replacing COM1IGV with IGV
Chiller 1: (33552, 13) (after 71034 zero points removed) writing to file data/BB1_chiller.csv 

replacing Lift with DTLift
no CH2Hz column for this chiller
replacing COM1IGV with IGV
Chiller 2: (36138, 13) (after 68448 zero points removed) writing to file data/BB2_chiller.csv 

replacing Lift with DTLift
no CH3Hz column for this ch

In [314]:
df.columns

Index(['timestamp', 'CH_kW/Ton', 'CH_kWh', 'CH1_CDW_DT', 'CH1_CHLRcmd_Alarm',
       'CH1_CHW_DT', 'CH1_CHWSTSP_Alarm', 'CH1_CompSH', 'CH1_CondApproach',
       'CH1_EvapApproach',
       ...
       'CH4SS', 'CH4STALLCNT', 'CH4SURGECNT', 'CH4Ton', 'CHkW',
       'CHLRShortCycle_Alarm', 'CHRSTG_ALARM', 'CHTON$25', 'commfail',
       'CommFail_Alarm'],
      dtype='object', length=208)

In [332]:
df.to_csv('BCBS_all.csv')

In [378]:
def ReadFiles(dat_dir, PlantCode, Chillers, CH_features):
    print(f'Plant {dat_dir} with code {PlantCode}')
    df = pd.DataFrame()
    for file in glob.glob('data/'+dat_dir+'/dss_'+dat_dir+'_chillers*.csv'):
        print(f'opening file: {file}')
        df = df.append(pd.read_csv(file))

    df = df.sort_values(by=['timestamp'])

    print('total data points:', df.shape,'\n')
    print('removing points where Load=0,kW/Ton=0 and kW/Ton>1','\n')

    my_file = Path('data/'+PlantCode+Chillers[0]+'_chiller.csv')
    print(my_file)
    if my_file.is_file() is False:
        print(CH_features)
        print(df.columns[0:60])        

    # remove rows where S (state) changes from 0 in previous row to 1 in this row (chiller start)
    data_size = df.shape[0]
    for c in Chillers:
        df = df[df['CH'+c+'SBoolean'].diff() != 1]
        df = df[df['CH'+c+'SBoolean'].diff() != -1]
    print(f'Data points removed for Startup and Shutdown of chiller: {data_size-df.shape[0]}')

    data_size = df.shape[0]
    for c in Chillers:
        try:
            mask = (df['CH'+c+'ALARM'] == 0)
            df = df[mask]
        except:
            print('no CH'+c+'ALARM point')
    print('just dropped CH with no Alarm \n',df.head())        
    print(f'Data points removed for Alarm of chiller: {data_size-df.shape[0]}')
 
    df_chiller = chiller_data(df,'1',CH_features)
    
    for c in Chillers:
        df_chiller = chiller_data(df,c,CH_features)
        #print(f'data points for chiller {c}:',df_chiller.shape)
        data_size = df_chiller.shape[0]
        df_chiller = df_chiller[df_chiller['Load']   != 0]
        df_chiller = df_chiller[df_chiller['kW/Ton'] != 0]
        df_chiller = df_chiller[df_chiller['kW/Ton'] < 1]

        #print(f'data points for chiller {c} after removing zeros:',df_chiller.shape)
        
        print(f'Chiller {c}: {df_chiller.shape} (after {data_size-df_chiller.shape[0]} zero points removed)',
             'writing to file data/'+PlantCode+c+'_chiller.csv','\n')

        df_chiller.to_csv('data/'+PlantCode+c+'_chiller.csv')
    return df

In [303]:
# extract just the chiller features of interest from the dataframe
# CH_features = list of features
# CH = chiller number
# df = dataframe with all the data
def chiller_data(df, CH, CH_features):
    df_chiller = pd.DataFrame()
    df_chiller['datetime'] = df['timestamp'].apply(get_datetime)
    
    for f in CH_features:
        if f[0] == '_':
            ff = f[1:]
        else:
            ff = f
        if f == 'Lift':
            print('replacing Lift with DTLift')
            ff = 'DTLift'
        if f == 'COM1IGV':
            print('replacing COM1IGV with IGV')
            ff = 'IGV'
        if f == 'COM1Hz':
            print('replacing COM1Hz with Hz')
            ff = 'Hz'
        
        feature = 'CH'+CH+f
        if feature in df.columns:
            df_chiller[ff] = df[feature]
        else:
            print(f'no {feature} column for this chiller')
            df_chiller[ff] = np.nan

    return df_chiller

def get_datetime(y):
    x = parse(y)
    return x

## Samford Data

Chillers 1-4 are **1500 Ton** variable speed YK chillers   

In [368]:
dat_dir = 'Samford'

CH_features = ['_kW/Ton','Load','CDWRT','CDWST','DTLift','Ton','_EvapApproach',
               '_CondApproach','Hz','_CompSH','IGV','REFLVL']

Chillers = ['1','2','3','4']

PlantCode = 'S'

df = ReadFiles(dat_dir, PlantCode, Chillers, CH_features)

Plant Samford with code S
opening file: data/Samford/dss_Samford_chillers.csv
total data points: (32807, 260) 

removing points where Load=0,kW/Ton=0 and kW/Ton>1 

data/S1_chiller.csv
Data points removed for Startup and Shutdown of chiller: 166
Data points removed for Alarm of chiller: 0
Chiller 1: (14172, 13) (after 18469 zero points removed) writing to file data/S1_chiller.csv 

Chiller 2: (5436, 13) (after 27205 zero points removed) writing to file data/S2_chiller.csv 

Chiller 3: (3792, 13) (after 28849 zero points removed) writing to file data/S3_chiller.csv 

Chiller 4: (13678, 13) (after 18963 zero points removed) writing to file data/S4_chiller.csv 



In [92]:
df.columns[4:53]

Index(['CH1_CDW_DT', 'CH1_CHLRcmd_Alarm', 'CH1_CHW_DT', 'CH1_CHWSTSP_Alarm',
       'CH1_CompSH', 'CH1_CondApproach', 'CH1_DP_Lift', 'CH1_DT_Lift',
       'CH1_EvapApproach', 'CH1_kW/Ton', 'CH1ALARM', 'CH1AVAIL', 'CH1CDP',
       'CH1CDT', 'CH1CDWRT', 'CH1CDWST', 'CH1CDWVLV', 'CH1CHWDP', 'CH1CHWFLO',
       'CH1CHWFLO_Meter', 'CH1CHWRT', 'CH1CHWST', 'CH1CHWSTSP', 'CH1CHWVLV',
       'CH1COMDISCHT', 'CH1CondApproach', 'CH1CYCLF', 'CH1DMD', 'CH1DPLift',
       'CH1DTLift', 'CH1EvapApproach', 'CH1EVP', 'CH1EVT', 'CH1F', 'CH1FLA',
       'CH1Hz', 'CH1IGV', 'CH1kW', 'CH1Lift', 'CH1Load', 'CH1REFLVL', 'CH1S',
       'CH1SBoolean', 'CH1SPD', 'CH1SS', 'CH1STATE', 'CH1SURGECNT', 'CH1Ton',
       'CH1WARNF'],
      dtype='object')

In [40]:
df_chiller.head()

,datetime,kW/Ton,Load,CDWRT,CDWST,DTLift,Ton,EvapApproach,CondApproach,Hz,CompSH,IGV,REFLVL
3064,2018-01-11 15:20:00+00:00,0.341737,0.489655,76.400002,69.500000,32.500000,734.482326,0.400002,0.199997,44.740002,9.500000,94.260002,15.700000
3065,2018-01-11 15:25:00+00:00,0.318592,0.324344,74.000000,69.099998,27.500000,486.516164,0.099998,-0.099998,43.349998,11.299995,3.930000,20.299999
3066,2018-01-11 15:30:00+00:00,0.356950,0.448242,75.300003,68.500000,30.900002,672.362777,0.400002,0.199997,45.740002,10.400002,14.240000,26.200001
3067,2018-01-11 15:35:00+00:00,0.349066,0.467915,75.400002,68.400002,30.900002,701.872192,0.500000,0.199997,45.340000,9.900002,20.629999,26.000000
3068,2018-01-11 15:40:00+00:00,0.353166,0.487023,75.800003,68.599998,31.400002,730.534315,0.500000,0.099998,45.139999,9.699997,32.029999,26.400000


## BCBS Data   

CH1 YKKJKLH9-CWF 900 Tons   
CH2 YKKJKLH9-CWF 900 Tons   
CH3 YKKJKLH9-CWF 900 Tons   
CH4 YKKCKSH9-CWG 900 Tons   

In [369]:
dat_dir = 'BCBS'

CH_features = ['_kW/Ton','Load','CDWRT','CDWST','DTLift','Ton','_EvapApproach',
               '_CondApproach','COM1Hz','_CompSH','IGV','REFLVL']

Chillers = ['1','2','3','4']

PlantCode = 'B'

df = ReadFiles(dat_dir, PlantCode, Chillers, CH_features)

Plant BCBS with code B
opening file: data/BCBS/dss_BCBS_chillers 2017-07.csv
opening file: data/BCBS/dss_BCBS_chillers 2017-11.csv
opening file: data/BCBS/dss_BCBS_chillers 2017-05.csv
opening file: data/BCBS/dss_BCBS_chillers 2017-01.csv
opening file: data/BCBS/dss_BCBS_chillers 2017-03.csv
opening file: data/BCBS/dss_BCBS_chillers 2017-09.csv
total data points: (105112, 208) 

removing points where Load=0,kW/Ton=0 and kW/Ton>1 

data/B1_chiller.csv
Data points removed for Startup and Shutdown of chiller: 1036
Data points removed for Alarm of chiller: 202
replacing COM1Hz with Hz
replacing COM1Hz with Hz
Chiller 1: (33388, 13) (after 70486 zero points removed) writing to file data/B1_chiller.csv 

replacing COM1Hz with Hz
Chiller 2: (35785, 13) (after 68089 zero points removed) writing to file data/B2_chiller.csv 

replacing COM1Hz with Hz
Chiller 3: (29500, 13) (after 74374 zero points removed) writing to file data/B3_chiller.csv 

replacing COM1Hz with Hz
Chiller 4: (23000, 13) (aft

In [188]:
df.to_csv('BCBS_all.csv')

In [114]:
df.columns[0:52]

Index(['timestamp', 'CH_kW/Ton', 'CH_kWh', 'CH1_CDW_DT', 'CH1_CHLRcmd_Alarm',
       'CH1_CHW_DT', 'CH1_CHWSTSP_Alarm', 'CH1_CompSH', 'CH1_CondApproach',
       'CH1_EvapApproach', 'CH1_kW/Ton', 'CH1ALARM', 'CH1AVAIL', 'CH1CDP',
       'CH1CDT', 'CH1CDWDP', 'CH1CDWRT', 'CH1CDWST', 'CH1CDWVLV', 'CH1CHWDP',
       'CH1CHWFLO', 'CH1CHWRT', 'CH1CHWRTS', 'CH1CHWST', 'CH1CHWSTS',
       'CH1CHWSTSP', 'CH1CHWVLV', 'CH1COM1F', 'CH1COM1Hz', 'CH1COM1IGV',
       'CH1COM1S', 'CH1COM1SPD', 'CH1COMDISCHT', 'CH1DMD', 'CH1DPLift',
       'CH1DTLift', 'CH1EVP', 'CH1EVT', 'CH1F', 'CH1FLA', 'CH1IGV', 'CH1kW',
       'CH1Lift', 'CH1Load', 'CH1REFLVL', 'CH1REFLVLS', 'CH1S', 'CH1SBoolean',
       'CH1SS', 'CH1STALLCNT', 'CH1SURGECNT', 'CH1Ton'],
      dtype='object')

In [ ]:
data_file = 'data/C2_chiller.csv'
CH1 = pd.read_csv(data_file)

data_file = 'data/C2_chiller.csv'
CH2 = pd.read_csv(data_file)

data_file = 'data/C3_chiller.csv'
CH3 = pd.read_csv(data_file)

data_file = 'data/C4_chiller.csv'
CH4 = pd.read_csv(data_file)

## Children's Mercy Hospital Data

CH1 YKK5K4K1-DAG 1200 Ton   
CH2 YKK5K4K1-DAG 1200 Ton   
CH3 YKK5K4K1-DAG 1200 Ton   
CH4 YKK5K4K1-DAG 1200 Ton   

In [370]:
dat_dir = 'CMH'

CH_features = ['_kW/Ton','Load','CDWRT','CDWST','DTLift','Ton','_EvapApproach',
               '_CondApproach','Hz','_CompSH','IGV','REFLVL']

Chillers = ['1','2','3','4']

PlantCode = 'C'

df = ReadFiles(dat_dir, PlantCode, Chillers, CH_features)

Plant CMH with code C
opening file: data/CMH/dss_CMH_chillers 2018-04.csv
opening file: data/CMH/dss_CMH_chillers 2018-01.csv
opening file: data/CMH/dss_CMH_chillers 2017-07.csv
opening file: data/CMH/dss_CMH_chillers 2017-10.csv
total data points: (85801, 195) 

removing points where Load=0,kW/Ton=0 and kW/Ton>1 

data/C1_chiller.csv
Data points removed for Startup and Shutdown of chiller: 484
Data points removed for Alarm of chiller: 16
Chiller 1: (36196, 13) (after 49105 zero points removed) writing to file data/C1_chiller.csv 

Chiller 2: (34809, 13) (after 50492 zero points removed) writing to file data/C2_chiller.csv 

Chiller 3: (46084, 13) (after 39217 zero points removed) writing to file data/C3_chiller.csv 

Chiller 4: (45241, 13) (after 40060 zero points removed) writing to file data/C4_chiller.csv 



In [213]:
data_file = 'data/C1_chiller.csv'
CH1 = pd.read_csv(data_file)

data_file = 'data/C2_chiller.csv'
CH2 = pd.read_csv(data_file)

data_file = 'data/C3_chiller.csv'
CH3 = pd.read_csv(data_file)

data_file = 'data/C4_chiller.csv'
CH4 = pd.read_csv(data_file)

In [215]:
print(CH1.describe().loc['50%'][7:],'\n')
print(CH2.describe().loc['50%'][7:],'\n')
print(CH3.describe().loc['50%'][7:],'\n')
print(CH4.describe().loc['50%'][7:])

EvapApproach     3.400002
CondApproach     4.000000
Hz              53.500000
CompSH          11.400002
IGV             87.000000
REFLVL          50.099998
Name: 50%, dtype: float64 

EvapApproach     3.200001
CondApproach     4.000000
Hz              53.099998
CompSH          12.500000
IGV             90.000000
REFLVL          55.000000
Name: 50%, dtype: float64 

EvapApproach     2.599998
CondApproach     4.399994
Hz              50.799999
CompSH          12.400002
IGV             81.000000
REFLVL          63.200001
Name: 50%, dtype: float64 

EvapApproach     2.700001
CondApproach     2.400002
Hz              48.799999
CompSH          12.099998
IGV             88.000000
REFLVL          58.200001
Name: 50%, dtype: float64


In [218]:
CH1['CompSH'].mean()

10.730764759927014

In [217]:
CH1.describe()

,Unnamed: 0,kW/Ton,Load,CDWRT,CDWST,DTLift,Ton,EvapApproach,CondApproach,Hz,CompSH,IGV,REFLVL
count,36415.000000,36415.000000,34979.000000,36401.000000,36414.000000,36414.000000,36415.000000,36415.000000,36402.000000,36399.000000,36415.000000,36414.000000,36414.000000
mean,11736.534340,0.399063,0.788001,80.712398,71.285522,38.419212,959.001742,3.417682,3.986677,50.405879,10.730765,80.121821,49.943827
std,7718.971044,0.106203,0.154774,10.810009,10.337773,12.459820,194.144771,0.444035,1.063595,7.342910,2.381775,20.894514,5.832968
min,0.000000,0.121534,0.190322,46.500000,43.400002,-1.500000,228.385849,-5.600002,-13.900002,0.000000,-0.699997,0.000000,6.500000
25%,4755.500000,0.342331,0.742331,74.599998,64.699997,31.199997,896.467546,3.200001,3.300003,48.599998,9.799995,72.000000,46.000000
50%,10733.000000,0.437762,0.821919,84.800003,74.800003,43.400002,993.474625,3.400002,4.000000,53.500000,11.400002,87.000000,50.099998
75%,17852.500000,0.475863,0.896377,88.599998,79.099998,47.500000,1087.128562,3.599998,4.699997,55.200001,12.199997,95.000000,53.700001
max,26495.000000,0.932632,0.999933,102.800003,92.500000,62.000004,1497.102556,15.700001,15.699997,60.000000,26.500000,100.000000,99.800003


In [216]:
CH2.describe()

,Unnamed: 0,kW/Ton,Load,CDWRT,CDWST,DTLift,Ton,EvapApproach,CondApproach,Hz,CompSH,IGV,REFLVL
count,35045.000000,35045.000000,25168.000000,35043.000000,35043.000000,35043.000000,35045.000000,35045.000000,35045.000000,35043.000000,35045.000000,35043.000000,35043.000000
mean,12090.962448,0.348112,0.820847,79.440910,70.642730,37.201912,1065.818573,3.261778,3.946606,49.599817,11.581253,84.863482,54.393382
std,8570.417809,0.101916,0.167531,11.201707,10.843581,12.762953,215.947146,0.468408,1.307294,7.967486,2.728610,17.129981,6.635923
min,0.000000,0.100544,0.091297,47.500000,43.299999,-0.500000,109.555868,-8.799999,-23.099998,0.000000,-5.099998,0.000000,16.700001
25%,3632.000000,0.288048,0.696730,70.599998,60.599998,27.099998,986.531667,3.000000,3.199997,46.599998,10.300003,79.000000,50.599998
50%,11619.000000,0.383723,0.890644,84.199997,75.000000,42.999996,1128.005293,3.200001,4.000000,53.099998,12.500000,90.000000,55.000000
75%,21211.000000,0.421721,0.950578,88.000000,79.300003,46.999996,1211.899411,3.500000,4.799995,55.000000,13.200005,97.000000,58.900002
max,26495.000000,0.730471,0.999998,101.099998,92.400002,59.099998,1584.618255,7.700001,11.000000,60.000000,60.900002,100.000000,89.599998


In [198]:
CH3.describe()

,Unnamed: 0,kW/Ton,Load,CDWRT,CDWST,DTLift,Ton,EvapApproach,CondApproach,Hz,CompSH,IGV,REFLVL
count,46343.000000,46343.000000,41277.000000,46342.000000,46342.000000,46342.000000,46343.000000,46343.000000,46342.000000,46342.000000,46342.000000,46342.000000,46342.000000
mean,12063.412899,0.417712,0.738408,76.756361,68.237510,33.959697,926.865228,2.675094,4.386757,46.974406,11.510949,71.253809,63.186026
std,7690.462779,0.126279,0.218623,12.257224,11.314933,14.181552,274.117324,0.698621,0.777600,8.665967,3.118845,30.672811,5.587365
min,0.000000,0.127232,0.143019,37.299999,32.700001,-8.100002,171.622437,-4.700001,-4.099998,0.000000,3.400002,0.000000,32.099998
25%,6028.000000,0.296794,0.536360,64.400002,55.400002,20.200001,706.573621,2.199997,3.899998,39.799999,9.000000,56.000000,59.200001
50%,10840.000000,0.459702,0.831146,80.099998,71.400002,38.499996,1025.162364,2.599998,4.399994,50.799999,12.400002,81.000000,63.200001
75%,18638.000000,0.520222,0.911553,87.300003,77.800003,46.100002,1128.279129,3.199997,4.800003,53.799999,13.699997,97.000000,67.699997
max,26496.000000,0.948220,0.999988,112.199997,94.000000,70.500000,1562.243143,14.400002,16.599998,60.000000,29.500000,100.000000,99.800003


In [199]:
CH4.describe()

,Unnamed: 0,kW/Ton,Load,CDWRT,CDWST,DTLift,Ton,EvapApproach,CondApproach,Hz,CompSH,IGV,REFLVL
count,45490.000000,45490.000000,43362.000000,45490.000000,45490.000000,45490.000000,45490.000000,45490.000000,45490.000000,45490.000000,45490.000000,45490.000000,45490.000000
mean,11341.826138,0.399809,0.716488,75.982873,67.434269,33.124421,878.553431,2.753755,2.470224,45.136947,11.095234,79.030248,58.567723
std,7368.724074,0.137949,0.188886,12.453972,11.618791,14.332729,237.318192,0.493831,0.923418,8.664688,2.911253,27.058727,5.503857
min,158.000000,0.061458,0.192411,49.799999,46.900002,5.899998,230.892822,-2.299999,-6.199997,30.000000,-0.400002,0.000000,9.900000
25%,4558.000000,0.251819,0.550678,62.700001,54.099998,17.900002,678.552913,2.400002,1.799999,36.400002,8.400002,78.000000,54.799999
50%,11146.500000,0.433380,0.774953,79.099998,69.800003,37.200001,939.817476,2.700001,2.400002,48.799999,12.099998,88.000000,58.200001
75%,18059.750000,0.521133,0.864309,87.300003,77.900002,46.199997,1054.624350,3.000000,3.000000,52.500000,13.199997,98.000000,62.200001
max,25817.000000,0.824073,0.999999,101.800003,92.300003,61.100002,1527.460144,5.799999,12.399994,60.000000,29.399994,100.000000,94.000000


## Tabreed DB01

In [379]:
dat_dir = 'TabDB01'

CH_features = ['_kW/Ton','Load','CDWRT','CDWST','Lift','Ton','_EvapApproach',
               '_CondApproach','Hz','_CompSH','COM1IGV','REFLVL']

Chillers = ['1','2','3','4','5','6','7','8','9','10','11','12']

PlantCode = '1T'


df = ReadFiles(dat_dir, PlantCode, Chillers, CH_features)

Plant TabDB01 with code 1T
opening file: data/TabDB01/dss_TabDB01_chillers 2017-07.csv
opening file: data/TabDB01/dss_TabDB01_chillers 2017-11.csv
opening file: data/TabDB01/dss_TabDB01_chillers 2017-05.csv
opening file: data/TabDB01/dss_TabDB01_chillers 2017-01.csv


/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


opening file: data/TabDB01/dss_TabDB01_chillers 2017-03.csv
opening file: data/TabDB01/dss_TabDB01_chillers 2017-09.csv
total data points: (104854, 356) 

removing points where Load=0,kW/Ton=0 and kW/Ton>1 

data/1T1_chiller.csv
Data points removed for Startup and Shutdown of chiller: 1488
no CH6ALARM point
just dropped CH with no Alarm 
    CH10ALARM  CH10AMPS    CH10CDP    CH10CDT  CH10CDWRT  CH10CDWST  \
0        0.0       0.0  75.300003  72.800003  78.800003  76.800003   
1        0.0       0.0  75.300003  72.800003  78.800003  76.900002   
2        0.0       0.0  75.300003  72.800003  78.800003  76.800003   
3        0.0       0.0  75.199997  72.800003  78.800003  76.800003   
4        0.0       0.0  75.199997  72.800003  78.800003  76.800003   

   CH10CHWFLO  CH10CHWRT  CH10CHWST  CH10CHWSTSP            ...             \
0         0.0  68.400002  66.699997          0.0            ...              
1         0.0  68.500000  66.699997          0.0            ...              
2   

In [357]:
df.to_csv('DB05_all.csv')

## Tabreed DB05

In [372]:
dat_dir = 'TabDB05'

CH_features = ['_kW/Ton','Load','CDWRT','CDWST','Lift','Ton','EvapApproach',
               '_CondApproach','Hz','_CompSH','COM1IGV','REFLVL']

#Chillers = ['14','16','18','20']
Chillers = ['1','2','3','4']

PlantCode = '5T'


df = ReadFiles(dat_dir, PlantCode, Chillers, CH_features)

Plant TabDB05 with code 5T
opening file: data/TabDB05/dss_TabDB05_chillers 2017-10.csv
opening file: data/TabDB05/dss_TabDB05_chillers 2017-07.csv
opening file: data/TabDB05/dss_TabDB05_chillers 2018-01.csv


/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


opening file: data/TabDB05/dss_TabDB05_chillers 2018-04.csv
total data points: (79463, 191) 

removing points where Load=0,kW/Ton=0 and kW/Ton>1 

data/5T1_chiller.csv
Data points removed for Startup and Shutdown of chiller: 154
Data points removed for Alarm of chiller: 381
replacing Lift with DTLift
no CH1Hz column for this chiller
replacing COM1IGV with IGV
no CH1COM1IGV column for this chiller
replacing Lift with DTLift
no CH1Hz column for this chiller
replacing COM1IGV with IGV
no CH1COM1IGV column for this chiller
Chiller 1: (49761, 13) (after 29167 zero points removed) writing to file data/5T1_chiller.csv 

replacing Lift with DTLift
no CH2Ton column for this chiller
no CH2Hz column for this chiller
replacing COM1IGV with IGV
no CH2COM1IGV column for this chiller
Chiller 2: (20651, 13) (after 58277 zero points removed) writing to file data/5T2_chiller.csv 

replacing Lift with DTLift
no CH3Ton column for this chiller
no CH3Hz column for this chiller
replacing COM1IGV with IGV
no 

In [361]:
df.to_csv('DB05_all.csv')

## BioCork

In [373]:
dat_dir = 'BioCork'

CH_features = ['_kW/Ton','Load','CDWRT','CDWST','Lift','Ton','_EvapApproach',
               '_CondApproach','Hz','_CompSH','COM1IGV','REFLVL']

Chillers = ['1','2']

PlantCode = 'BC'


ReadFiles(dat_dir, PlantCode, Chillers, CH_features)

Plant BioCork with code BC
opening file: data/BioCork/dss_BioCork_chillers 2017-07.csv
opening file: data/BioCork/dss_BioCork_chillers 2017-01.csv


/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


opening file: data/BioCork/dss_BioCork_chillers 2018-01.csv
total data points: (132933, 78) 

removing points where Load=0,kW/Ton=0 and kW/Ton>1 

data/BC1_chiller.csv
Data points removed for Startup and Shutdown of chiller: 606
Data points removed for Alarm of chiller: 19723
replacing Lift with DTLift
no CH1Hz column for this chiller
replacing COM1IGV with IGV
no CH1REFLVL column for this chiller
replacing Lift with DTLift
no CH1Hz column for this chiller
replacing COM1IGV with IGV
no CH1REFLVL column for this chiller
Chiller 1: (68599, 13) (after 44005 zero points removed) writing to file data/BC1_chiller.csv 

replacing Lift with DTLift
no CH2Hz column for this chiller
replacing COM1IGV with IGV
no CH2REFLVL column for this chiller
Chiller 2: (37569, 13) (after 75035 zero points removed) writing to file data/BC2_chiller.csv 



,CH1ALARM,CH1AVAIL,CH1CDP,CH1CDT,CH1CDWRT,CH1CDWST,CH1CDWVLV,CH1CHWDP,CH1CHWFLO,CH1CHWRT,...,CH2_CDW_DT,CH2_CHW_DT,CH2_CompSH,CH2_CondApproach,CH2_EvapApproach,CH2_kW/Ton,CH2kW,COMLOSSBAS,commfail,timestamp
0,0.0,1.0,73.215042,72.500000,68.720001,64.400002,0.0,7.541960,1757.324951,45.680000,...,0.719997,1.800003,10.440002,-3.959999,-4.860001,0.0,0.0,1.0,1.0,2017-03-19T01:00:00.000Z
1,0.0,1.0,66.615822,66.739998,64.580002,60.259998,0.0,7.541960,1760.494995,44.779999,...,0.719997,1.800003,10.440002,-3.959999,-4.860001,0.0,0.0,1.0,1.0,2017-03-19T01:05:00.000Z
2,0.0,1.0,69.705132,68.540001,64.760002,60.259998,0.0,7.469441,1779.673950,44.779999,...,0.719997,1.800003,10.440002,-3.959999,-4.860001,0.0,0.0,1.0,1.0,2017-03-19T01:10:00.000Z
3,0.0,1.0,74.708931,72.320000,68.540001,64.040001,0.0,7.469441,1769.054199,44.779999,...,0.719997,1.800003,10.440002,-3.959999,-4.860001,0.0,0.0,1.0,1.0,2017-03-19T01:15:00.000Z
4,0.0,1.0,72.417336,72.500000,68.900002,63.680000,0.0,7.541960,1769.054199,45.139999,...,0.719997,1.800003,10.440002,-3.959999,-4.860001,0.0,0.0,1.0,1.0,2017-03-19T01:20:00.000Z
5,0.0,1.0,66.311249,67.459999,62.959999,59.180000,0.0,7.541960,1756.294678,45.139999,...,0.719997,1.800003,10.440002,-3.959999,-4.860001,0.0,0.0,1.0,1.0,2017-03-19T01:25:00.000Z
6,0.0,1.0,68.704369,67.459999,62.959999,59.180000,0.0,7.541960,1771.194092,45.139999,...,0.719997,1.800003,10.440002,-3.959999,-4.860001,0.0,0.0,1.0,1.0,2017-03-19T01:30:00.000Z
7,0.0,1.0,72.808937,70.879997,66.739998,62.779999,0.0,7.541960,1760.494995,44.599998,...,0.719997,1.800003,10.440002,-3.959999,-4.860001,0.0,0.0,1.0,1.0,2017-03-19T01:35:00.000Z
8,0.0,1.0,71.416573,70.879997,68.540001,64.760002,0.0,7.614479,1770.084473,44.599998,...,0.719997,1.800003,10.440002,-3.959999,-4.860001,0.0,0.0,1.0,1.0,2017-03-19T01:40:00.000Z
9,0.0,1.0,66.108192,66.559998,64.400002,58.820000,0.0,7.541960,1748.844971,44.599998,...,0.539997,1.800003,10.259998,-3.779999,-4.860001,0.0,0.0,1.0,1.0,2017-03-19T01:45:00.000Z


## Check consistency of files

In [279]:
for file in glob.glob('data/*chiller.csv'):
    print(f'opening file: {file}')
    df = pd.read_csv(file)
    print(df.shape)
    print(df.columns,'\n')


opening file: data/5T14_chiller.csv
(49819, 14)
Index(['Unnamed: 0', 'datetime', 'kW/Ton', 'Load', 'CDWRT', 'CDWST', 'DTLift',
       'Ton', 'EvapApproach', 'CondApproach', 'Hz', 'CompSH', 'IGV', 'REFLVL'],
      dtype='object') 

opening file: data/S3_chiller.csv
(3842, 14)
Index(['Unnamed: 0', 'datetime', 'kW/Ton', 'Load', 'CDWRT', 'CDWST', 'DTLift',
       'Ton', 'EvapApproach', 'CondApproach', 'Hz', 'CompSH', 'IGV', 'REFLVL'],
      dtype='object') 

opening file: data/1T8_chiller.csv
(61464, 14)
Index(['Unnamed: 0', 'datetime', 'kW/Ton', 'Load', 'CDWRT', 'CDWST', 'DTLift',
       'Ton', 'EvapApproach', 'CondApproach', 'Hz', 'CompSH', 'IGV', 'REFLVL'],
      dtype='object') 

opening file: data/1T12_chiller.csv
(64449, 14)
Index(['Unnamed: 0', 'datetime', 'kW/Ton', 'Load', 'CDWRT', 'CDWST', 'DTLift',
       'Ton', 'EvapApproach', 'CondApproach', 'Hz', 'CompSH', 'IGV', 'REFLVL'],
      dtype='object') 

opening file: data/1T1_chiller.csv
(54716, 14)
Index(['Unnamed: 0', 'datetime', 